In [ ]:
!pip install --upgrade moviepy
!git clone https://github.com/kenshohara/video-classification-3d-cnn-pytorch.git
!pip install torch torchvision torchaudio

Cloning into 'video-classification-3d-cnn-pytorch'...
remote: Enumerating objects: 121, done.
remote: Total 121 (delta 0), reused 0 (delta 0), pack-reused 121
Receiving objects: 100% (121/121), 158.63 KiB | 833.00 KiB/s, done.
Resolving deltas: 100% (63/63), done.


In [ ]:
import json
import os
from moviepy.editor import *
from moviepy import *

In [10]:
new_directory = '/content'
os.chdir(new_directory)

updated_directory = os.getcwd()
print("更新后的工作目录:", updated_directory)

更新后的工作目录: /content


In [11]:
def loadVideosToInfile1(folder):
    videos = os.listdir(folder)
    with open('output1', 'w') as inputfile:
        for v in videos:
            if v.split('.')[-1] == 'mp4':
                inputfile.write(v+'\n')
loadVideosToInfile1('input_films')

In [12]:
def loadVideosToInfile2(folder):
    videos = os.listdir(folder)
    with open('output2', 'w') as inputfile:
        for v in videos:
            if v.split('.')[-1] == 'mp4':
                inputfile.write(v+'\n')
loadVideosToInfile2('input_films2')

In [15]:
!python video-classification-3d-cnn-pytorch/main.py --input output1 --video_root input_films --output ../output_features1.json --model video-classification-3d-cnn-pytorch/resnet-34-kinetics-cpu.pth --mode feature --no_cuda


loading model video-classification-3d-cnn-pytorch/resnet-34-kinetics-cpu.pth
Traceback (most recent call last):
  File "/content/video-classification-3d-cnn-pytorch/main.py", line 24, in <module>
    model_data = torch.load(opt.model)
  File "/usr/local/lib/python3.10/dist-packages/torch/serialization.py", line 1028, in load
    return _legacy_load(opened_file, map_location, pickle_module, **pickle_load_args)
  File "/usr/local/lib/python3.10/dist-packages/torch/serialization.py", line 1264, in _legacy_load
    typed_storage._untyped_storage._set_from_file(
RuntimeError: unexpected EOF, expected 8276210 more bytes. The file might be corrupted.


In [16]:
!python video-classification-3d-cnn-pytorch/main.py --input output2 --video_root input_films2 --output ../output_features2.json --model video-classification-3d-cnn-pytorch/resnet-34-kinetics-cpu.pth --mode feature --no_cuda


loading model video-classification-3d-cnn-pytorch/resnet-34-kinetics-cpu.pth
Traceback (most recent call last):
  File "/content/video-classification-3d-cnn-pytorch/main.py", line 24, in <module>
    model_data = torch.load(opt.model)
  File "/usr/local/lib/python3.10/dist-packages/torch/serialization.py", line 1028, in load
    return _legacy_load(opened_file, map_location, pickle_module, **pickle_load_args)
  File "/usr/local/lib/python3.10/dist-packages/torch/serialization.py", line 1264, in _legacy_load
    typed_storage._untyped_storage._set_from_file(
RuntimeError: unexpected EOF, expected 8276210 more bytes. The file might be corrupted.


In [ ]:
file = open('../output_features2.json')
segments = json.load(file)
len(segments)

In [ ]:
for s in segments: print(s['video'])

In [ ]:
print(segments[0]['clips'][0].keys())

In [ ]:
import numpy as np
import json

In [ ]:
file1 = open('../output_features1')
segments1 = json.load(file1)
file2 = open('../output_features2')
segments2 = json.load(file2)

film1Features = []
for s in segments1[0]['clips']:
    film1Features.append(np.array(s['features']))

In [ ]:
def find_best_range(films, sample_film_features):
    best_range_info = {
        'start': 0,
        'score': 100000000,
        'filmX_index': -1
    }

    for filmX_index, filmX in enumerate(films):
        filmX_features = []
        for s in filmX['clips']:
            filmX_features.append(np.array(s['features']))

        for i in range(len(filmX_features) - len(sample_film_features)):
            distance = 0
            for j in range(len(sample_film_features)):
                d = np.linalg.norm(filmX_features[i + j] - sample_film_features[j])
                distance += d

            if distance < best_range_info['score']:
                best_range_info['start'] = i
                best_range_info['score'] = distance
                best_range_info['filmX_index'] = filmX_index

    return best_range_info

best_range_info = find_best_range(segments2, film1Features)
print(f"Best Range Start: {best_range_info['start']}")
print(f"Best Range Score: {best_range_info['score']}")
print(f"Best Range belongs to Film Index: {best_range_info['filmX_index']}")

In [ ]:
from IPython.display import Video

def exportXVideo(XVideo, Samfeatures, best_range_info):
    tsfps = XVideo.fps
    tsFrameLength = 1 / tsfps
    nrFrames = len(Samfeatures) * 16
    startFrame = best_range_info['start'] * 16
    startTime = startFrame * tsFrameLength
    endTime = startTime + nrFrames * tsFrameLength
    try:
        X_Matched_sample_Video = XVideo.subclip(startTime, endTime)
        X_Matched_sample_Video.ipython_display()
        X_Matched_sample_Video.write_videofile("output_films/X_Matched_sample_Video.mp4", audio_codec='aac')
    except Exception as e:
        print(f"Fail: {e}")
    return X_Matched_sample_Video

XVideo = VideoFileClip(r"../ScrapedVideos/Abramovic-Ulay_The-Great-Wall.mp4")
Samfeatures = film1Features
best_range_info = best_range_info
X_Matched_sample_Video = exportXVideo(XVideo, Samfeatures, best_range_info)

In [ ]:
X_Matched_sample_Video.duration

In [ ]:
frame = X_Matched_sample_Video.get_frame(1)

In [ ]:
from PIL import Image

In [ ]:
Image.fromarray(frame,'RGB')